In [1]:
from sklearn.metrics import accuracy_score,classification_report, precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import os
import numpy as np

In [2]:
import json
with open(f"GYAFC_test_labels.json") as f:
    list_label = json.load(f)

In [3]:
len(list_label)

41600

In [15]:
def highlight_max(s):
    if s.dtype == np.object:
        is_max = [False for _ in range(s.shape[0])]
    else:
        is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]
# https://www.geeksforgeeks.org/highlight-the-maximum-value-in-each-column-in-pandas/

In [5]:
def get_metrics(true_labels, predicted_labels):
        
    prec, rec, fscore, _ = precision_recall_fscore_support(true_labels, predicted_labels)
    
    acc = accuracy_score(true_labels, predicted_labels)
    
    fsc_macr = f1_score(true_labels, predicted_labels, average='macro')
    
    res_dict = {
                "acc": acc,
                "form_prec": prec[0],
               "form_rec": rec[0],
               "form_fscore": fscore[0],
        
                "inform_prec": prec[1],
                "inform_rec": rec[1],
                "inform_fscore": fscore[1],
                "fsc_macr": fsc_macr
               }
    
    return res_dict    

In [7]:
# 'microsoft_deberta-base_ep10_wus100_lr1e-06.json' in  models_index

In [18]:
paper_sota = {
                "acc": 0.864,
                "form_prec": 0.831,
               "form_rec": 0.985,
               "form_fscore": 0.88,
        
                "inform_prec": 0.965,
                "inform_rec": 0.807,
                "inform_fscore": 0.842,
               }
paper_sota["fsc_macr"] = (paper_sota["form_fscore"] + paper_sota["inform_fscore"])/2

In [19]:
all_models_folder = "test_results/"

models_index = []
data = []

exclude = ["microsoft_deberta-large_ep5_wus100_lr5e-05.json",
          "microsoft_deberta-large_ep5_wus100_lr0.0001.json",
          "microsoft_deberta-large_ep10_wus100_lr0.0001.json"]

data.append(paper_sota)
models_index.append("paper_sota")

for model_pred_path in os.listdir(all_models_folder):
    
    if "ipynb_checkpoints" not in model_pred_path and model_pred_path not in exclude:
    
        test_predicts_path = os.path.join(all_models_folder, model_pred_path)

        with open(test_predicts_path) as f:
            test_predicts_list = json.load(f)

        models_index.append(model_pred_path)


        res_dict_i = get_metrics(list_label, test_predicts_list)
        data.append(res_dict_i)

        # break
    
df_report = pd.DataFrame(data, index = models_index)
df_report = df_report.sort_values(by=["fsc_macr"], ascending=False)

df_report.style.apply(highlight_max)

/tmp/ipykernel_29156/3263289477.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if s.dtype == np.object:


,acc,form_prec,form_rec,form_fscore,inform_prec,inform_rec,inform_fscore,fsc_macr
microsoft_deberta-base_ep10_wus2000_lr0.0001.json,0.871779,0.836785,0.943163,0.886795,0.924307,0.790478,0.852170,0.869482
microsoft_deberta-base_ep5_wus1000_lr0.0001.json,0.868726,0.851842,0.912103,0.880944,0.891120,0.819322,0.853714,0.867329
microsoft_deberta-base_ep5_wus2000_lr1e-06.json,0.870361,0.824215,0.961627,0.887634,0.946052,0.766415,0.846812,0.867223
microsoft_deberta-base_ep10_wus2000_lr1e-06.json,0.870168,0.824260,0.961085,0.887430,0.945346,0.766620,0.846654,0.867042
microsoft_deberta-base_ep5_wus100_lr1e-06.json,0.869351,0.823352,0.960769,0.886768,0.944832,0.765232,0.845601,0.866185
microsoft_deberta-base_ep10_wus2000_lr1e-05.json,0.867861,0.828119,0.948761,0.884344,0.930033,0.775721,0.845897,0.865121
microsoft_deberta-base_ep10_wus100_lr1e-06.json,0.867596,0.822927,0.957338,0.885058,0.940307,0.765386,0.843878,0.864468
microsoft_deberta-base_ep5_wus2000_lr1e-05.json,0.866298,0.827057,0.946910,0.882935,0.927582,0.774487,0.844149,0.863542
microsoft_deberta-base_ep10_wus1000_lr5e-05.json,0.866058,0.829694,0.941763,0.882184,0.921614,0.779834,0.844817,0.863500
microsoft_deberta-base_ep10_wus2000_lr5e-05.json,0.865697,0.821806,0.954810,0.883330,0.936901,0.764204,0.841786,0.862558
